# Creating JSON from raw web exam page

In [315]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import json

In [377]:
url = "https://partner-academy.databricks.com/learn/course/1366/play/8578/test-your-knowledge"
username = "arthur.couget@viseo.com"
password = "Mathboy3232!"
json_name = "manage_data_access_with_unity_catalog"

In [378]:
page_source, questions = main(url, username, password, json_name)
questions

{0: {'questions': 'A metastore administrator has enabled identity federation for a Unity Catalog namespace.\nWhich of the following correctly describes the privileges that users who have access to the catalog now have? Select two responses.\n',
  'choices': ['The users can be assigned from the workspace to the account by an account administrator through the account console.',
   'The users can be assigned from the account to the workspace by the account administrators through the account console.',
   'The users can be assigned from the account to the workspace by the workspace administrator through their workspace administration console.',
   'The users can be assigned from the workspace to the account by the workspace administrator through their workspace administration console.',
   'The users can be assigned from the workspace to the account by a workspace user through the identity provider.'],
  'expected_answers': 2,
  'answers': []},
 1: {'questions': 'Which of the following des

# Get source page

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def spawn_driver(driver_path, url):
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options, executable_path=driver_path)
    driver.get(url)
    return driver

In [62]:
def get_source(url, username, password):
    driver = spawn_driver("chromedriver", url)
    time.sleep(20)
    uname = driver.find_element("xpath", "/html/body/div[1]/div/div/div/doc-layout/div/main/div[2]/doc-layout-external-simple/div/div[2]/div/doc-layout-login/div[1]/form/ui-input-text/div/div[1]/div[1]/input")
    uname.send_keys(username)
    pword = driver.find_element("xpath", "/html/body/div[1]/div/div/div/doc-layout/div/main/div[2]/doc-layout-external-simple/div/div[2]/div/doc-layout-login/div[1]/form/ui-input-password/div/div[1]/div[1]/input")
    pword.send_keys(password)
    button = driver.find_element("xpath", "/html/body/div[1]/div/div/div/doc-layout/div/main/div[2]/doc-layout-external-simple/div/div[2]/div/doc-layout-login/div[1]/form/div[1]/ui-button-raised-primary/ui-button-raised/button")
    button.click()
    time.sleep(10)
    startquiz = driver.find_element("xpath", "/html/body/div[1]/div/div/div/doc-layout/div/main/div[2]/ng-component/div/div[1]/div/div/div/div/div[1]/div/chapter-viewer/div/ui-trap/div/div/div[2]/div/div[3]/div/quiz-player/quiz-player-dummy/div/div/div[4]/rectangular-button/button")
    startquiz.click()
    time.sleep(10)
    page_source = driver.page_source
    return page_source

In [64]:
page_source = get_source(url, username, password)
page_source

'<html class="mdl-js" style="--hyd-color-accent-primary:#1b3139; --ui-color-accent-primary:#1b3139; --hyd-color-icon-accent:#fe5000; --ui-color-icon-accent-primary:#fe5000; --hyd-color-layout-brand:#1b3139; --ui-color-layout-brand:#1b3139; --hyd-color-button-label-accent:#618794; --ui-color-button-label-accent:#618794; --hyd-color-avatar-background-accent:#1b3139; --ui-color-avatar-background-accent:#1b3139; --hyd-color-interaction-hover-accent-primary:#1b3139; --ui-color-interaction-hover-accent-primary:#1b3139; --hyd-color-interaction-ripple-accent-primary:#1b3139; --ui-color-interaction-ripple-accent-primary:#1b3139; --hyd-color-loading-accent:#1b3139; --ui-color-loading-accent:#1b3139; --ui-color-dialog-background-accent:#1b3139; --hyd-color-border-accent-secondary:#fe5000; --ui-color-border-accent-secondary:#fe5000; --hyd-color-typography-accent:#fe5000; --ui-color-typography-accent:#fe5000; --hyd-color-typography-accent-secondary:#fe5000; --ui-color-typography-accent-secondary:#f

# Extract questions

In [349]:
def clean_question(question):
    question = question.replace("&nbsp;", "")
    question = question.replace("</strong>", "")
    question = question.replace("<strong>", "")
    question = question.replace("\xa0", "")
    question = question.replace('</span><br style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;"/><br style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;"/>', "")
    question = question.replace('</span><br style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;"/>', "")
    question = question.replace("Code block:", "")
    question = question.replace("</span>", "")
    question = question.replace('<div style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;">', "")
    question = question.replace('<div style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;">', "")
    question = question.replace("</span>", "")
    question = question.replace('<span style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;">', "")
    return question

def get_questions(page_source):
    i = 0
    dic_questions = {}
    soup = BeautifulSoup(page_source, 'html.parser')
    questions_raw = soup.find_all("div", {"class": "question-block default-headings-styles"})
    for question in questions_raw:
        question_str = str(question)
        question_list = re.findall(r'<p>(.*?)</p>', question_str)
        question_complete = "\n".join(question_list)
        question_complete = clean_question(question_complete)
        dic_questions[i]["question"] = question_complete
        dic_questions[i]["choices"] = get_answers(question_str)
        dic_questions[i]["answers"] = []
        i = i+1
    return dic_questions

In [353]:
def get_questions(page_source):
    i = 0
    dic_questions = {}
    soup = BeautifulSoup(page_source, 'html.parser')
    questions_raw = soup.find_all("div", {"class": "question-block default-headings-styles"})
    for question in questions_raw:
        question_str = str(question)
        question_list = re.findall(r'<p>(.*?)</p>', question_str)
        question_complete = "\n".join(question_list)
        question_complete = clean_question(question_complete)
        dic_questions[i] = {}
        dic_questions[i]["questions"] = question_complete
        dic_questions[i]["choices"] = get_answers(question_str)
        dic_questions[i]["expected_answers"] = get_expected_answers(question_complete)
        dic_questions[i]["answers"] = []
        i = i+1
    return dic_questions

In [366]:
questions = get_questions(page_source)
questions[15]

{'questions': 'A data engineer needs to query a JSON file whose location is represented by the variable path.\n\nWhich of the following commands do they need to use? Select one response.\n',
 'choices': ['RETURN json.`${path}`;',
  'SELECT * FROM json.`${path}`;',
  'SHOW TABLE json.`${path}`;',
  'SELECT * FROM path LOCATION `${path}`;',
  'DISPLAY TABLE json.`${path}`;'],
 'expected_answers': 1,
 'answers': []}

# Answers

In [352]:
def get_answers(question):
    answers = re.findall(r'text-grey-dark ng-star-inserted">(.*?)</span>', question)
    if len(answers)==0:
        answers = re.findall(r'style="visibility: visible;">(.*?)</span>', question)
    answers = list(map(lambda x: x.replace("<strong>", ""), answers))
    answers = list(map(lambda x: x.replace("</strong>", ""), answers))
    answers = list(map(lambda x: x.replace('''<span style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;">''', ""), answers))
    answers = list(map(lambda x: x.replace('''<div style="color:#222222;font-size:small;font-family:Arial, Helvetica, sans-serif;">''', ""), answers))
    return answers

In [301]:
def get_expected_answers(question):
    expected_str = question.split(' ')[-2]
    if expected_str == "one":
        expected_nb = 1
    elif expected_str == "two":
        expected_nb = 2
    elif expected_str == "three":
        expected_nb = 3
    elif expected_str == "four":
        expected_nb = 4
    else: expected_nb = 1
    return expected_nb

# Save to json

In [308]:
name = "manage_data_with_delta_lake"
with open("../questions/" + name + ".json", "w") as ftw:
    json.dump(questions, ftw)

In [310]:
def save_to_json(name, dic_questions):
    with open("../questions/" + name + ".json", "w") as ftw:
        json.dump(dic_questions, ftw)

# Main

In [321]:
def main(url, username, password, json_name):
    page_source = get_source(url, username, password)
    questions = get_questions(page_source)
    save_to_json(json_name, questions)
    return page_source, questions